<a href="https://colab.research.google.com/github/Chirag314/LLM/blob/main/LLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install -Uqqq rich openai tiktoken wandb tenacity

In [24]:
import tiktoken
import wandb
from pprint import pprint
from wandb.integration.openai import autolog
import random

In [25]:
import os
from getpass import getpass
import openai
from pathlib import Path
from rich.markdown import Markdown
import pandas as pd
from tenacity import(
    retry,
    stop_after_attempt,
    wait_random_exponential,
)


In [26]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['COLAB' in x for x in os.environ.keys()]):
    print('Please enter password in the prompt at the top of your window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

In [27]:
# For logging into W&B
autolog({"project":"llmapps","job_type":"introduction"})

usage/completion_tokens,▁▂█
usage/elapsed_time,▁▁█
usage/prompt_tokens,▁▁█
usage/total_tokens,▁▁█
usage/completion_tokens,358
usage/elapsed_time,4.6468
usage/prompt_tokens,218
usage/total_tokens,576


wandb: Currently logged in as: cdesai. Use `wandb login --relogin` to force relogin


GENERATING SYNTHETIC SUPPORT QUESTIONS

In [28]:
# ADD A RETRY BEHAVIOUR IN CASE WE HIT THE API RATE LIMIT
@retry(wait=wait_random_exponential(min=1,max=60),stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
  return openai.ChatCompletion.create(**kwargs)

In [29]:
model_name="gpt-3.5-turbo"

In [30]:
system_prompt="You are a helpful assistant"
user_prompt="Generate a support question from W & B user"

def generate_and_print(system_prompt,user_prompt,n=5):
  messages=[
      {"role":"system","content":system_prompt},
      {"role":"user","content":user_prompt},
  ]
  responses=completion_with_backoff(
      model=model_name,
      messages=messages,
      n=n

  )
  for response in responses.choices:
    generation =response.message.content
    display(Markdown(generation))

generate_and_print(system_prompt,user_prompt)

What steps can I take to troubleshoot a connectivity issue with my Wi-Fi network?

What steps should I follow if I want to update my payment information on the W & B platform?

How can I troubleshoot issues with my Wi-Fi connection?

"How do I format a table to display data in different colors for each cell in Microsoft Word?"

What steps should I take if I forgot my Wi-Fi password and need to connect a new device to the network?

In [31]:
#test if examples.txt is present, download it if not
if not Path("examples.txt").exists():
  !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

In [32]:
delimiter='\t'
with open('examples.txt','r') as file:
  data=file.read()
  real_queries=data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")
Markdown(f"Sample one :\n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one : "how do I utilize Weights & Biases LangChain integration>?"

We can now use those real user questions to guide our model to produce synthetic questions like those

In [33]:
def generate_few_shot_prompts(queries,n=3):
  prompt="Generate a support question from an W&B user\n"+\
        "Below you will find a few examples of real user queries"
  for _ in range(n):
    prompt +=random.choice(queries) + "\n"
  prompt +="Let's start!"
  return prompt

generation_prompt=generate_few_shot_prompts(real_queries)
Markdown(generation_prompt)

Generate a support question from an W&B user Below you will find a few examples of real user queriesI would like to
track the weights and gradients for my model while it trains. I have three networks that are being trained jointly.
I am calling ‘wandb.watch’ for each one of them individually. This allowed me to see the gradients, but for some   
reason, I can’t see the weights for one of these networks in the ‘parameters’ tab.                                 

I would appreciate if anyone could help me to figure out what is the issue and how I can fix it. can you tell me   
why if i log sns.heatmaps, it overwrites previous images? Hey , Is there a recommended way to use Launch in an     
SLURM environment? Let's start!

Lets check this

In [34]:
generate_and_print(system_prompt,user_prompt=generation_prompt)

What could be causing the weights of one of my networks to not be displayed in the 'parameters' tab after calling  
'wandb.watch' for each of them individually?

Why can't I see the weights for one of my networks in the 'parameters' tab while using 'wandb.watch' for each      
network separately?                                                                                                

How can I prevent logging sns.heatmaps from overwriting previous images?                                           

What is the recommended way to use Launch in an SLURM environment?

I'm sorry, but as an AI assistant, I do not have access to real-time user queries. However, I can help you with any
questions or issues you have regarding W&B. Please let me know how I can assist you!

Why can't I see the weights for one of my networks in the 'parameters' tab when using 'wandb.watch'? How can I fix 
this issue?                                                                                                        

Why does logging sns.heatmaps overwrite previous images? Is there a way to prevent this?                           

What is the recommended way to use Launch in an SLURM environment?

Why am I unable to see the weights for one of my networks in the 'parameters' tab when using 'wandb.watch' for each
network individually?

Add context and response

#Check if directory exists, if not create it and downlaod the samples
if not os.path.exists("../notebooks"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample/notebooks ../

In [51]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

In [52]:
def find_md_files(directory):
  "Find all markdown files in a directory and return their contents"
  md_files=[]
  for file in Path(directory).rglob("*.md"):
    with open(file,'r',encoding='utf-8') as md_file:
      content=md_file.read()
    md_files.append((file.relative_to(directory),content))
    return md_files

documents=find_md_files('../docs_sample/')
len(documents)

1

In [53]:
#LEts check if documents are not too long for our context window
tokenizer=tiktoken.encoding_for_model(model_name)
tokens_per_document=[len(tokenizer.encode(document)) for _,document in documents]
pprint(tokens_per_document)

[956]


Some of them are too long , extract random chunk from a document



In [54]:
def extract_random_chunk(document,max_tokens=512):
  tokens=tokenizer.encode(document)
  if len(tokens)<=max_tokens:
    return document
  start=random.randint(0,len(tokens)-max_tokens)
  end=start+max_tokens
  return tokenizer.decode(tokens[start:end])

Now we will use that extracted chunk to create a question tat can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [55]:
def generate_context_prompt(chunk):
  prompt = "Generate a support question from a W&B user\n"+\
  "The question should be answerable by provided fragment of W&B documentation.\n" +\
  "Below you will find a fragment of W&B documenatation:\n"+\
  chunk + "\n"+\
  "Let's start!"
  return prompt

chunk=extract_random_chunk(documents[0][1])
generation_prmopt=generate_context_prompt(chunk)


In [56]:
Markdown(generation_prmopt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documenatation: ', value: 'projectpage'}, {label: 'Run Page', 
value: 'runpage'}, ]}>                                                                                             

After a run is created, you can update tags using ]8;id=938971;../../../guides/track/public-api-guide.md\our public API]8;;\ like so:                                          

                                                                                                                   
 run = wandb.Api().run("{entity}/{project}/{run-id}"})                                                             
 run.tags.append("tag1")  # you can choose tags based on run data here                                             
 run.update()                                                                                                      
                                                                                                                   

You can read more about how to use the Public API in the ]8;id=386414;../../../ref/README.md\reference documentation]8;;\ or ]8;id=351263;../../../guides/track/public-api-guide.md\guide]8;;\.                         


This method is best suited to tagging large numbers of runs with the same tag or tags.                             

In the ]8;id=85914;../pages/project-page.md#search-for-runs\runs sidebar]8;;\ of the ]8;id=302930;../pages/project-page.md\Project Page]8;;\,  click the table icon in the upper-right.  This will expand the sidebar   
into the full ]8;id=586131;runs-table.md\runs table]8;;\.                                                                                          

Hover over a run in the table to see a checkbox on the left or look in the header row for a checkbox that will     
allow you to select all runs.                                                                                      

Click the checkbox to enable bulk actions. Select the runs to which you'd like to apply your tag(s).               

Click the Tag button above the rows of runs.                                                                       

Type a tag you'd like to add and click "Add" below the text box to add a new tag.                                  


This method is best suited to applying a tag or tags to a single run by hand.                                      

In the left sidebar of the ]8;id=787025;../pages/run-page.md\Run Page]8;;\, click the top ]8;id=517027;../pages/run-page.md#overview-tab\Overview tab]8;;\.                                                   

Next to "Tags" is a gray ➕ button. Click on that plus to add a tag.                                               

Type a tag you'd like to add and click "Add" below the text box to add a new tag.                                  



                                                How to remove tags                                                 

Tags can also be removed from runs via the UI.                                                                     


<Tabs defaultValue="projectpage" values={[ {label: 'Project Page', value: 'projectpage'}, {label: 'Run Page',      
value: 'runpage'}, ]}>                                                                                             

This method is best suited to removing tags from a Let's start!

In [43]:
generate_and_print(system_prompt,generation_prompt,n=3)

What could be the reason that I am unable to see the weights for one of my networks in the 'parameters' tab when   
using 'wandb.watch'? How can I fix this issue?

Support Question: Why are the weights not visible for one of my networks in the 'parameters' tab when using        
'wandb.watch' for each network individually during training? How can I fix this issue?

What could be the reason why I cannot see the weights for one of my networks in the 'parameters' tab even though I 
called 'wandb.watch' for each one individually?

LEVEL 5 PROMPT
COMPLEX DIRECTIVE THAT INCLUDE:
1. Description of high level prompt
2. A detailed bulleted list of sub-tasks.
3. An explicit statement asking LLM to explain its own output
4. A guidance on how LLM output will be evaluated
5. Few shot examples

In [80]:
# we will use GPT4 model
model_name='gpt-3.5-turbo'

In [57]:
#test if examples.txt is present, download it if not
if not Path("system_template.txt").exists():
  !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/system_template.txt

--2023-08-30 22:59:10--  https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/system_template.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 469 [text/plain]
Saving to: ‘system_template.txt’

system_template.txt 100%[===================>]     469  --.-KB/s    in 0s      

2023-08-30 22:59:10 (22.1 MB/s) - ‘system_template.txt’ saved [469/469]



In [59]:
# read system_template.txt file
with open("system_template.txt","r") as file:
  system_prompt=file.read()

In [60]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [62]:
#test if examples.txt is present, download it if not
if not Path("prompt_template.txt").exists():
  !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/prompt_template.txt

--2023-08-30 23:01:51--  https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/prompt_template.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1054 (1.0K) [text/plain]
Saving to: ‘prompt_template.txt’

prompt_template.txt 100%[===================>]   1.03K  --.-KB/s    in 0s      

2023-08-30 23:01:51 (38.2 MB/s) - ‘prompt_template.txt’ saved [1054/1054]



In [63]:
#read prompt_template.txt into an f-string
with open("prompt_template.txt","r") as file:
  prompt_template=file.read()

In [64]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [65]:
def generate_context_prompt(chunk,n_questions=3):
  questions = '\n'.join(random.sample(real_queries,n_questions))
  user_prompt=prompt_template.format(QUESTIONS=questions,CHUNK=chunk)
  return user_prompt
user_prompt=generate_context_prompt(chunk)

In [66]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
I have an config file apart from sweep_config file so when I try to run sweep agent, I am getting                  
UnboundLocalError("local variable 'config' referenced before assignment"). what is the capital of Ireland? Can we  
start multiple long-running WANDB runs at the same time, withinin the same project?                                

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

', value: 'projectpage'}, {label: 'Run Page', value: 'runpage'}, ]}>                                               

After a run is created, you can update tags using ]8;id=137140;../../../guides/track/public-api-guide.md\our public API]8;;\ like so:                                          

                                                                                                                   
 run = wandb.Api().run("{entity}/{project}/{run-id}"})                                                             
 run.tags.append("tag1")  # you can choose tags based on run data here                                             
 run.update()                                                                                                      
                                                                                                                   

You can read more about how to use the Public API in the ]8;id=834288;../../../ref/README.md\reference documentation]8;;\ or ]8;id=242147;../../../guides/track/public-api-guide.md\guide]8;;\.                         


This method is best suited to tagging large numbers of runs with the same tag or tags.                             

In the ]8;id=124720;../pages/project-page.md#search-for-runs\runs sidebar]8;;\ of the ]8;id=222455;../pages/project-page.md\Project Page]8;;\,  click the table icon in the upper-right.  This will expand the sidebar   
into the full ]8;id=201684;runs-table.md\runs table]8;;\.                                                                                          

Hover over a run in the table to see a checkbox on the left or look in the header row for a checkbox that will     
allow you to select all runs.                                                                                      

Click the checkbox to enable bulk actions. Select the runs to which you'd like to apply your tag(s).               

Click the Tag button above the rows of runs.                                                                       

Type a tag you'd like to add and click "Add" below the text box to add a new tag.                                  


This method is best suited to applying a tag or tags to a single run by hand.                                      

In the left sidebar of the ]8;id=989717;../pages/run-page.md\Run Page]8;;\, click the top ]8;id=967583;../pages/run-page.md#overview-tab\Overview tab]8;;\.                                                   

Next to "Tags" is a gray ➕ button. Click on that plus to add a tag.                                               

Type a tag you'd like to add and click "Add" below the text box to add a new tag.                                  



                                                How to remove tags                                                 

Tags can also be removed from runs via the UI.                                  

In [81]:
def generate_questions(documents,n_questions=3,n_generations=5):
  questions=[]
  for _, document in documents:
    chunk=extract_random_chunk(document)
    user_prompt=generate_context_prompt(chunk,n_questions)
    messages=[
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt},
    ]
    response=completion_with_backoff(
        model=model_name,
        messages=messages,
        n=n_generations
    )
    questions.extend([response.choices[i].message.content for i in range(n_generations)])
  return questions




In [82]:
# function to parse model generation and extract context, question and answer

def parse_generation(generation):
  lines=generation.split("\n")
  context=[]
  question=[]
  answer=[  ]
  flag=None

  for line in lines:
    if "CONTEXT:" in line:
      flag='context'
      line=line.replace("CONTEXT:","").strip()
    elif "QUESTION:" in line:
      flag = "question"
      line = line.replace("QUESTION:", "").strip()
    elif "ANSWER:" in line:
      flag = "answer"
      line = line.replace("ANSWER:", "").strip()

    if flag=='context':
      context.append(line)
    elif flag=='question':
      question.append(line)
    elif flag=='answer':
      answer.append(line)
  context='\n'.join(context)
  question='\n'.join(question)
  answer='\n'.join(answer)

  return context,question,answer


In [83]:
generations=generate_questions([documents[0]],n_questions=3,n_generations=5)


In [84]:
parse_generation(generations[0])

('The user is working on a machine learning project and is using Weights & Biases (W&B) to track and visualize their experiments. They have multiple hyperparameters to explore using sweeps, but they want to avoid certain combinations of hyperparameters that may cause GPU memory issues. The user is wondering if there is a way to prune unwanted combinations of hyperparameters during a sweep in W&B.\n',
 'How can I prune unwanted combinations of hyperparameters during a sweep in Weights & Biases?\n',
 'Yes, you can prune unwanted combinations of hyperparameters during a sweep in Weights & Biases. While there is not a built-in feature similar to optuna\'s special exception to cancel a run, you can achieve a similar result by using tags in Weights & Biases. You can define a threshold for a specific metric, and if a combination of hyperparameters results in a value below the threshold, you can add a specific tag to that run. Then, you can filter out or ignore runs with that tag when analyzin

In [85]:
parsed_generations=[]
generations=generate_questions(documents,n_questions=3,n_generations=5)

for generation in generations:
  context, question,answer=parse_generation(generation)
  parsed_generations.append({"context":context,"question":question,"answer":answer})

  #convert to pandas dataframe and save locally
  df=pd.DataFrame(parsed_generations)
  df.to_csv('generated_examples.csv',index=False)

  #log df as a table to W&B for interactive exploration
  wandb.log({'generated_examples': wandb.Table(dataframe=df)})

  #log csv file as an artifact
  artifact=wandb.Artifact("generated_examples",type='dataset')
  artifact.add_file("generated_examples.csv")
  wandb.log_artifact(artifact)

In [86]:
wandb.finish()

usage/completion_tokens,▁▂▁█▆
usage/elapsed_time,▁▁▁█▆
usage/prompt_tokens,▁▂▂██
usage/total_tokens,▁▂▁█▆
usage/completion_tokens,1488
usage/elapsed_time,9.35255
usage/prompt_tokens,1036
usage/total_tokens,2524
